<a href="https://colab.research.google.com/github/jungwonsherryzang/Cloud-based-Web-Application-with-ADS-B-real-time-data/blob/main/clean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#!pip3 install flightradar24
#!pip3 install --upgrade google-cloud
#!pip3 install --upgrade google-cloud-storage
#!pip3 install --upgrade google-cloud-bigquery
#!pip install --upgrade pandas_gbq
#!pip install pyarrow
#!pip install mysql-connector-python

In [25]:
#!pip3 install geopandas
#!pip3 install geoplot

In [5]:
from google.cloud import bigquery
import mysql.connector
import pandas as pd
import json
import flightradar24
import requests
import math
import os
from datetime import datetime
import geopandas as gpd
import geoplot as gplt
import folium
from branca.element import Figure
fr = fr = flightradar24.Api()

# **Python Code for Populating OpenSky Data onto Google Cloud**

Find Airport Information

In [17]:
def findAirportLoc(name):
    #FlightRadar24
    dataAirport = fr.get_airports()
    listAirport= dataAirport['rows']
    for airport in listAirport:
        if(name in airport['name']):
            return airport['lat'], airport['lon']

Set the distance around airport for OpenSky data

In [7]:
def findLocRange(lat_center,long_center):
    #20km range
    range = 0.09
    lat_min = lat_center - range
    lat_max = lat_center + range
    long_min = long_center - (range/ math.cos(lat_center*math.pi/180))
    long_max = long_center + (range / math.cos(lat_center*math.pi/180))
    return lat_min,lat_max,long_min,long_max

Queries Opensky network with given lat and lon information and returns data as pandas dataframe

In [8]:
def collectData(lat_min, lat_max, long_min, long_max):
    r = requests.get('https://opensky-network.org/api/states/all?lamin='+lat_min+'&lomin='+long_min+'&lamax='+lat_max+'&lomax='+long_max)
    r.status_code
    file=r.json()

    df=pd.DataFrame(file['states'],columns=['icao24', 'callsign','origin_country','time_position','last_contact',
                                            'longitude','latitude','baro_altitude','on_ground','velocity',
                                            'true_track','vertical_rate','sensors','geo_altitude','squawk','spi','x','y']).drop(columns=['x','y']).assign(obs_time= lambda x: file['time'])
    df[['time_position','last_contact','obs_time']] = df[['time_position','last_contact','obs_time']].apply(pd.to_datetime, unit='s')
    #df.set_index(['obs_time', 'callsign'],inplace=True)
    return df

Cleans Pandas DataFrame

In [9]:
def cleanDataframe(df):
    df = df.drop(columns=['sensors','squawk'])
    df = df.where(pd.notnull(df), None)
    df = df.reset_index(drop=True)
    return df

Pushes cleaned DataFrame to the cloud

In [18]:
def connectQuery(df):
    credentials = r'https://github.com/jungwonsherryzang/Cloud-based-Web-Application-with-ADS-B-real-time-data/blob/main/cs540-project-key.json'
    os.environ['Google_APPLICATION_CREDENTIALS'] = credentials
    client = bigquery.Client()
    table_id = 'cs540-project.Flights.AllData'
    df = df.to_gbq(table_id , if_exists='append')

Test Purposes Only

In [19]:
def createTest(): #Test the database on the test table
    credentials = r'https://github.com/jungwonsherryzang/Cloud-based-Web-Application-with-ADS-B-real-time-data/blob/main/cs540-project-key.json'
    os.environ['Google_APPLICATION_CREDENTIALS'] = credentials
    client = bigquery.Client()
    table_id = 'cs540-project.Flights.Test'
    rows_to_insert = [
        {u'Name' : 'Mike', u'Age': 23},
        {u'Name' : 'DummyData', u'Age': 24},
        ]
    errors = client.insert_rows_json(table_id,rows_to_insert)

Main function which starts process

In [12]:
def main():
    
    lat_center, long_center = findAirportLoc('Daytona')
    lat_min, lat_max, long_min, long_max = findLocRange(lat_center, long_center)
    df = collectData(str(lat_min), str(lat_max), str(long_min), str(long_max))
    df = cleanDataframe(df)
    #connectQuery(df)
    return(df)
    
adsb_df = main()

In [23]:
adsb_df

,icao24,callsign,origin_country,time_position,last_contact,longitude,latitude,baro_altitude,on_ground,velocity,true_track,vertical_rate,geo_altitude,spi,obs_time
0,abadf9,SWA2384,United States,2022-04-11 01:15:40,2022-04-11 01:15:40,-81.1336,29.1053,3505.20,False,177.62,159.31,-9.10,3649.98,False,2022-04-11 01:15:40
1,ab2786,EPI262,United States,2022-04-11 01:15:39,2022-04-11 01:15:39,-81.0765,29.1732,68.58,False,44.42,61.65,4.23,99.06,False,2022-04-11 01:15:40
2,ab3506,ERU821,United States,2022-04-11 01:14:31,2022-04-11 01:14:34,-81.0589,29.1753,-22.86,False,32.23,65.48,-3.58,15.24,False,2022-04-11 01:15:40
3,ac05a6,ERU874,United States,2022-04-11 01:15:39,2022-04-11 01:15:39,-81.0750,29.1638,137.16,False,42.83,335.90,-1.30,182.88,False,2022-04-11 01:15:40
4,a1e395,N22010,United States,2022-04-11 01:13:24,2022-04-11 01:13:24,-81.0592,29.1751,-22.86,False,32.45,64.65,-1.30,7.62,False,2022-04-11 01:15:40
5,a65c8f,JED185,United States,2022-04-11 01:14:12,2022-04-11 01:14:12,-81.0777,29.1727,-30.48,False,34.48,88.29,-3.25,7.62,False,2022-04-11 01:15:40


Build Map

In [29]:
fig=Figure(width=550,height=350)
m=folium.Map(width=550,height=350,location=[29.18, -81.05],zoom_start=11,min_zoom=8,max_zoom=14)
fig.add_child(m)
icon_url=r'https://github.com/jungwonsherryzang/Cloud-based-Web-Application-with-ADS-B-real-time-data/blob/main/plane_icon.png'
for i in range(0,len(adsb_df)):
    lat = adsb_df.loc[i]['latitude']
    lon = adsb_df.loc[i]['longitude']
    callsign = adsb_df.loc[i]['callsign']
    new_icon = folium.features.CustomIcon(icon_url, icon_size=(16,16), icon_anchor=('center'))
    folium.Marker([lat, lon], popup=callsign, icon=new_icon).add_to(m)
m